In [27]:
def load_dataset():
    return [[1,3,4] , [2,3,5] , [1,2,3,5], [2,5]]

In [28]:
from numpy import *

In [29]:
load_dataset()

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [30]:
def createC1(dataset):
    c1 = []
    for transaction in  dataset:
        for item in transaction:
            if not [item] in c1:
                c1.append([item])
    c1.sort()
    return list(map(frozenset,c1)) #Frozen Sets can be declared as keys for dictionaries but not list

In [31]:
def scanD(dataset,candidates,min_support):
    sscnt = {}
    for tid in dataset:
        for can in candidates:
            if can.issubset(tid):
                sscnt.setdefault(can,0)
                sscnt[can] +=1
    num_items = float(len(dataset))
    retlist = []
    support_data = {}
    for key in sscnt:
        support = sscnt[key]/num_items
        if support >= min_support:
            retlist.insert(0,key)
        support_data[key] = support
    return retlist, support_data

In [32]:
dataset = load_dataset()

In [33]:
c1 = createC1(dataset) #Returns candidates
c1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [34]:
D = list(map(set,dataset))

In [35]:
L1 , SD = scanD(D,c1,0.6)

In [36]:
L1

[frozenset({3}), frozenset({2}), frozenset({5})]

In [37]:
SD

{frozenset({4}): 0.25,
 frozenset({5}): 0.75,
 frozenset({2}): 0.75,
 frozenset({3}): 0.75,
 frozenset({1}): 0.5}

In [38]:
def aprioriGen(Lk,k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        L1 = list(Lk[i])[:k-2]
        L1.sort()
        for j in range(i+1,lenLk):
            L2 = list(Lk[j])[:k-2]
            L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j])
    return retList
    

In [39]:
Ck = aprioriGen(L1,2)

In [40]:
Ck

[frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5})]

In [41]:
L1

[frozenset({3}), frozenset({2}), frozenset({5})]

In [42]:
def apriori(dataset, minSupport= 0.60):
    C1 = createC1(dataset)
    D = list(map(set,dataset))
    L1 , supportData = scanD(D,C1,minSupport)
    L = [L1]
    k = 2
    while(len(L[k-2])>0):
        Ck = aprioriGen(L[k-2],k)
        Lk ,supK = scanD(D,Ck,minSupport)
        supportData.update(supK)
        L.append(Lk)
        k+=1
    return L[:-1], supportData

    

In [43]:
apriori(dataset)

([[frozenset({3}), frozenset({2}), frozenset({5})], [frozenset({2, 5})]],
 {frozenset({5}): 0.75,
  frozenset({3}): 0.75,
  frozenset({3, 5}): 0.5,
  frozenset({4}): 0.25,
  frozenset({2, 3}): 0.5,
  frozenset({2, 5}): 0.75,
  frozenset({1}): 0.5,
  frozenset({2}): 0.75})

In [44]:
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = 
        supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print (freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

In [45]:
def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            print(freqSet)
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList   

In [46]:
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [47]:
L,suppData= apriori(dataset,minSupport=0.7)
print(L)
print(suppData)

[[frozenset({3}), frozenset({2}), frozenset({5})], [frozenset({2, 5})]]
{frozenset({5}): 0.75, frozenset({3}): 0.75, frozenset({3, 5}): 0.5, frozenset({4}): 0.25, frozenset({2, 3}): 0.5, frozenset({2, 5}): 0.75, frozenset({1}): 0.5, frozenset({2}): 0.75}


In [48]:
rules= generateRules(L,suppData, minConf=0.7)


frozenset({2, 5})
frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
